https://github.com/sje30/dl2021/blob/main/dla1_2021.pdf


[2] https://www.simplilearn.com/tutorials/deep-learning-tutorial/perceptron

[1] https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.461.5174&rep=rep1&type=pdf

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import math

# Seed for reproducability
random.seed(5401767)

In [ ]:
# Function to create binary samples with labels
def generateSamples(options = [-1,1], n_values = 10, n_samples = 100, classifier_type = "sum"):
    # Randomly generate data
    data = np.array([[random.choice(options) for i in range(n_values)] for j in range(n_samples)])
    if (classifier_type == "sum"):
        # Assign binary labels, where 1 is positive sum and 0 is negative sum
        labels = np.array([int(sum(x) >= 0) for x in data])
    else:
        # Assign binary labels, where 1 is positive sign product and 0 is negative sign
        labels = np.array([math.prod(x) for x in data])
        labels = np.where(labels == -1, 0, labels)
    return(data, labels)

In [ ]:
# Generate training, testing and validation data
x_train_sum, y_train_sum = generateSamples(n_samples = 1000)
x_val_sum, y_val_sum = generateSamples(n_samples = 100)
x_test_sum, y_test_sum = generateSamples(n_samples = 100)

x_train_product, y_train_product = generateSamples(n_samples = 1000, classifier_type = "product")
x_val_product, y_val_product = generateSamples(n_samples = 100, classifier_type = "product")
x_test_product, y_test_product = generateSamples(n_samples = 100, classifier_type = "product")

In [ ]:
class Perceptron():
    """
    n_neurons: the size of the input layer
    bias_weight: weight to initialise bias unit
    learning_rate: step size to change weights
    epochs: number of training iterations
    activation_function: use step or sigmoid activation function
    """
    def __init__(self, n_neurons, bias_weight = 0, learning_rate = 0.1, activation_function = "sigmoid"):
        # Set parameters
        self.n_neurons = n_neurons
        self.bias_weight = bias_weight
        self.learning_rate = learning_rate
        self.activation_function = activation_function
        # Initialise network weights and bias
        self.weights = self.initialiseWeights()
        
    def initialiseWeights(self):
        # Randomly initialise weights from a normal distribution
        random_weights = np.random.randn(self.n_neurons)
        # Add the bias weight to index zero
        weights = np.append(self.bias_weight, random_weights)
        return(weights)
        
    def shuffleData(self, data, labels):
        # Add data into a dataframe
        df = pd.DataFrame(data = data)
        df["label"] = labels
        # Shuffle rows
        df = df.sample(frac = 1)
        # Extract data and labels
        shuffled_data = np.array(df.loc[:, df.columns != "label"])
        shuffled_labels = np.array(df["label"])
        return(shuffled_data, shuffled_labels)
    
    def predict(self, data, labels):
        # Record network predictions for each value
        predictions = np.array([])
        
        for value in data:
            # Dot product of the weights and input vector values plus bias
            output = sum(self.weights[1:] * value) + self.weights[0]
            
            # Apply activation function
            if (self.activation_function.lower() == "sigmoid"):
                # Sigmoid activation function
                prediction = 1/(1 + math.exp(-output))
            else:
                # Step activation function
                prediction = int(output > 0)
            predictions = np.append(predictions, prediction)
            
        return(predictions)
        
    def train(self, train_data, train_labels, val_data = None, val_labels = None,
              epochs = 100, shuffle = True):
        
        # Record error over epochs
        all_train_mse, all_val_mse = list(), list()
        
        # Run for a specified number of epochs
        for epoch in range(epochs):
            if (shuffle):
                # Shuffle the order of the training data
                train_data, train_labels = self.shuffleData(train_data, train_labels)
                
            # Record training and validation error for the epoch
            train_error = 0
            val_error = 0
            
            # Make predictions over all data
            predictions = self.predict(train_data, train_labels)
                
            # Iterate over predictions, samples and labels
            for pred, value, label in zip(predictions, train_data, train_labels):
                # For each sample, update the model weights
                weight_update = self.learning_rate * (label - pred)
                self.weights[0] += weight_update
                self.weights[1:] += weight_update * value
                
                # Add square of the differences between the predicted and expected output
                train_error += (pred - label)**2
                
            # Check if validation data given
            if (not isinstance(val_data, type(None))):
                # Make predictions on validation data
                val_predictions = self.predict(val_data, val_labels)
                # Record the validation loss
                for pred, label in zip(val_predictions, val_labels):
                    val_error += (pred - label)**2
                    
                # Calculate validation MSE
                all_val_mse.append(val_error/len(val_data))
            
            # Calculate mean squared error (MSE) for training data
            all_train_mse.append(train_error/len(train_data))
            
            # Stop training if train MSE is 0
            if all_train_mse[len(all_train_mse)-1] == 0:
                break
                
        return(all_train_mse, all_val_mse)
    
    def calculateAccuracy(self, truth, prediction):
        # Accuracy between expected and predicted outputs
        accuracy = np.sum(np.equal(truth, prediction)) / len(truth)
        return(accuracy)

In [ ]:
# Initialise the perceptrons
perceptron_sum = Perceptron(n_neurons = 10, learning_rate = 0.01, activation_function = "step")
perceptron_product = Perceptron(n_neurons = 10, learning_rate = 0.01, activation_function = "step")
# Train the perceptrons
train_error_sum, val_error_sum = perceptron_sum.train(x_train_sum, y_train_sum, x_val_sum, y_val_sum,
                                                      epochs = 50)
train_error_product, val_error_product = perceptron_product.train(x_train_product, y_train_product,
                                                                  x_val_product, y_val_product,
                                                                  epochs = 50)

In [ ]:
# Create plot of training error over epochs
fig, axes = plt.subplots(1, 2)
# Adjust space between subplots
plt.subplots_adjust(right = 1.5, wspace = 0.3)

# Add data
axes[0].plot(train_error_sum, label = 'Training')
axes[0].plot(val_error_sum, label = 'Validation')
axes[0].title.set_text('A. Binary Sum Sign Training Error')
axes[1].plot(train_error_product, label = 'Validation')
axes[1].plot(val_error_product, label = 'Training')
axes[1].title.set_text('B. Binary Product Parity Training Error')

# Set labels
for ax in axes.flat:
    ax.set(xlabel = 'Epoch', ylabel = 'MSE')
    
# Add legend, save and display plot
plt.legend(bbox_to_anchor = (1.45, 1.05))
plt.savefig("Perceptron/training_error.pdf", bbox_inches = 'tight')
plt.show()

In [ ]:
# Calculate accuracy on test set
y_pred_sum = perceptron_sum.predict(x_test_sum, y_test_sum)
y_pred_product = perceptron_sum.predict(x_test_product, y_test_product)
print("Binary sum sign accuracy: " + str(perceptron_sum.calculateAccuracy(y_test_sum, y_pred_sum) * 100) + "%")
print("Binary product parity accuracy: " + str(perceptron_product.calculateAccuracy(y_test_product, y_pred_product) * 100) + "%")